In [7]:
%%writefile query.hql

USE stackoverflow_;


WITH questions AS (
    SELECT year, tags_expl, COUNT(*) AS cnt
    FROM posts
    LATERAL VIEW explode(tags) tagsTable AS tags_expl
    WHERE post_type_id = 1 AND year IN (2009, 2016)
    GROUP BY year, tags_expl
)
SELECT concat_ws("\t", tags_expl, string(rank_2016), string(rank_2009), string(cnt_2016), string(cnt_2009))
FROM (
    SELECT
        t_2016.tags_expl AS tags_expl
        ,t_2016.rank AS rank_2016
        ,t_2009.rank AS rank_2009
        ,t_2016.cnt AS cnt_2016
        ,t_2009.cnt AS cnt_2009
    FROM (
        SELECT tags_expl, cnt, RANK() OVER (ORDER BY cnt DESC) rank
        FROM questions
        WHERE year = 2016
        LIMIT 10
    ) t_2016
    LEFT OUTER JOIN (
        SELECT tags_expl, cnt, RANK() OVER (ORDER BY cnt DESC) rank
        FROM questions
        WHERE year = 2009
    ) t_2009
    ON t_2016.tags_expl = t_2009.tags_expl
    ORDER BY rank_2016
) t_top;

Overwriting query.hql


In [8]:
! hive -f query.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.155 seconds
Query ID = jovyan_20180527103939_0803f688-d551-4d89-82ec-cc4c6ebdd14d
Total jobs = 8
Launching Job 1 out of 8
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1527414825805_0012, Tracking URL = http://c9bfde1d3c8a:8088/proxy/application_1527414825805_0012/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1527414825805_0012
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2018-05-27 10:40:04,626 Stage-1 map = 0%,  reduce = 0%
2018-05-27 10:40:14,471 Stage-1 map = 100%, 

javascript	1	5	2771	192
java	2	2	2033	243
android	3	52	1809	25
php	4	3	1673	215
python	5	11	1585	108
c#	6	1	1519	423
html	7	14	1212	84
jquery	8	8	1167	141
ios	9	186	914	7
css	10	20	801	59
Time taken: 252.085 seconds, Fetched: 10 row(s)
